In [1]:
!pip install pandas openpyxl scikit-learn

import pandas as pd
from sklearn.ensemble import IsolationForest

# Step 1: Read the Excel file
file_path = r"C:\SIFE_GESMES_M_SERIES.xlsx"
df = pd.read_excel(file_path)

# Step 2: Convert DATA_TYPE column to string
df['DATA_TYPE'] = df['DATA_TYPE'].astype(str)

# Step 3: Create key column
df['key'] = df[['COUNT_AREA', 'IVF_REP_SECTOR', 'IVF_ITEM', 'MATURITY_ORIG', 'DATA_TYPE', 'BS_COUNT_SECTOR', 'CURRENCY_TRANS']].astype(str).agg('.'.join, axis=1)

# Step 4: Add iteration column
df['iteration'] = df.groupby('key').ngroup() + 1  # Number groups starting from 1

# Step 5: Define a function to apply Isolation Forest to each group
def apply_isolation_forest(group):
    iso = IsolationForest()
    if len(group) > 1:  # Ensure there are enough points to fit the model
        group['score'] = iso.fit_predict(group[['OBS_VALUE']])
        group['score'] = iso.score_samples(group[['OBS_VALUE']])  # Get the actual score
        group['outlier_indication'] = group['score'].apply(lambda x: 'outlier' if x < -0.7 else 'inlier')
    else:
        group['score'] = None
        group['outlier_indication'] = 'incomplete'
    return group

# Step 6: Apply the function to each group
df = df.groupby('key').apply(apply_isolation_forest).reset_index(drop=True)

# Step 7: Select and reorder columns for the final output
df_final = df[['COUNT_AREA', 'COUNT_AREA_DESC', 'IVF_REP_SECTOR', 'IVF_REP_SECTOR_DESC', 
               'IVF_ITEM', 'IVF_ITEM_DESC', 'MATURITY_ORIG', 'MATURITY_ORIG_DESC', 
               'DATA_TYPE', 'DATA_TYPE_DESC', 'BS_COUNT_SECTOR', 'BS_COUNT_SECTOR_DESC', 
               'CURRENCY_TRANS', 'CURRENCY_TRANS_DESC', 'TIME_PERIOD', 'OBS_VALUE', 
               'OBS_CONF', 'OBS_STATUS', 'key', 'iteration', 'score', 'outlier_indication']]

# Step 8: Write the final DataFrame to an Excel file
output_file_path = r"C:\SIFE_GESMES_M_SERIES_output.xlsx"
df_final.to_excel(output_file_path, index=False)

print(f"Data successfully written to {output_file_path}")


Defaulting to user installation because normal site-packages is not writeable
Data successfully written to L:\Ecoreturn\IFs\Power BI\IF Balance Sheet Monthly Analysis\SIFE_GESMES_M_SERIES_k_ALIDA2.xlsx
